In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd

In [2]:
Data = pd.read_csv("./Source/CarOR_VanAvailability_LSOA_AnalysisCombined.csv",index_col=['PowerLiftRatio,Perc','Lower layer Super Output Areas Code'])
# CSV file taken from 'https://www.ons.gov.uk/datasets/TS045/editions/2021/versions/4/filter-outputs/21f5ef3b-68bf-44ee-b610-f654febc6e8d#get-data'

In [3]:
Data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1284192 entries, (100, 'E01000001') to (170, 'W01002040')
Data columns (total 11 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   1 car or van in household            1284192 non-null  int64  
 1   2 cars or vans in household          1284192 non-null  int64  
 2   3 or more cars or vans in household  1284192 non-null  int64  
 3   No cars or vans in household         1284192 non-null  int64  
 4   TotalHousehold                       1284192 non-null  int64  
 5   TotalCars                            1284192 non-null  int64  
 6   PowerProfile_kWe                     1284192 non-null  object 
 7   NumberOfCars                         1284192 non-null  object 
 8   TotalCarsCharged                     1284192 non-null  float64
 9   EV_SufficiencyFlag                   1284192 non-null  object 
 10  EV_ChargingPenetrationRate           1

In [4]:
DropColumns = ['1 car or van in household', '2 cars or vans in household','3 or more cars or vans in household', 'No cars or vans in household',
                'TotalHousehold', 'TotalCars', 'PowerProfile_kWe', 'NumberOfCars','TotalCarsCharged', 'EV_SufficiencyFlag']
Data.drop(columns=DropColumns,inplace=True)

In [5]:
Data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1284192 entries, (100, 'E01000001') to (170, 'W01002040')
Data columns (total 1 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   EV_ChargingPenetrationRate  1284192 non-null  float64
dtypes: float64(1)
memory usage: 19.2 MB


In [6]:
SHAPEFILE = './ShapeFiles/Lower_layer_Super_Output_Areas_2021_EW_BFE_V9_-9107090204806789093/LSOA_2021_EW_BFE_V9.shp'
GeoDF = gpd.read_file(SHAPEFILE)[['LSOA21CD', 'geometry']]
SHAPEFILE = 0
GeoDF.columns = ['LSOA_21_Code', 'geometry']
GeoDF.head()

,LSOA_21_Code,geometry
0,E01000001,"POLYGON ((532151.537 181867.433, 532152.500 18..."
1,E01000002,"POLYGON ((532634.497 181926.016, 532632.048 18..."
2,E01000003,"POLYGON ((532153.703 182165.155, 532158.250 18..."
3,E01000005,"POLYGON ((533619.062 181402.364, 533639.868 18..."
4,E01000006,"POLYGON ((545126.852 184310.838, 545145.213 18..."


In [7]:
GeoDF.info(memory_usage='deep')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35672 entries, 0 to 35671
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   LSOA_21_Code  35672 non-null  object  
 1   geometry      35672 non-null  geometry
dtypes: geometry(1), object(1)
memory usage: 2.5 MB


In [11]:
# EVPenetrationRateBins = np.arange(0,101,10)
# s = Data.loc[120]["EV_ChargingPenetrationRate"].value_counts(bins=EVPenetrationRateBins,sort=False)

In [12]:
# sdf = pd.DataFrame(s).transpose()
# sdf

In [13]:
def PlotFigure(DF,UpLift):
    Title = f'EV charging penetration LSOA\nPower Uplift of +{UpLift-100}%.'
    col = 'EV_ChargingPenetrationRate'
    vmin = DF[col].min()
    vmax = DF[col].max()
    cmap = 'RdYlGn'
    fig, ax = plt.subplots(1,figsize=(9,16),dpi=1000)
    ax.axis('off')
    DF.plot(column=col, ax=ax, edgecolor='white', linewidth=0.01, cmap=cmap)
    ax.set_title(Title, fontdict={'fontsize': '14', 'fontweight': '3'})# Add a title
    sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=0, vmax=100), cmap=cmap)# Create colorbar as a legend
    # Empty array for the data range
    sm._A = []
    cbaxes = fig.add_axes([0.18,0.35,0.02,0.2])
    cbar = fig.colorbar(sm, cax=cbaxes)
    plt.savefig(f"./Pictures/EV_Charging_Penetration_LSOA_{UpLift}.png")
    plt.close('all')

In [14]:
PowerLifts = set()
for item in Data.index:
    PowerLifts.add(item[0])
PowerLifts = sorted(PowerLifts)

In [26]:
for ChargeUpLift in tqdm(PowerLifts[0::5]):
    SelectedData = Data.loc[ChargeUpLift].reset_index()
    MergedDF = pd.merge(left=GeoDF, right=SelectedData, how='left', left_on='LSOA_21_Code', right_on='Lower layer Super Output Areas Code')
    DropColumns = ['Lower layer Super Output Areas Code']
    MergedDF.drop(columns=DropColumns,inplace=True)
    PlotFigure(MergedDF,ChargeUpLift)

100%|████████████████████████████████████████████████████████████████████| 7/7 [03:25<00:00, 29.31s/it]
